In [ ]:
from pathlib import Path

data_dir = Path("../data/preprocessed_train_data")
audio_dir = data_dir / "audio"

audio_path = sorted(audio_dir.glob("*.pt"))[0]
audio_path

PosixPath('../data/preprocessed_train_data/audio/5a03d20a7ecfc50001be0a7a_q1_generic.pt')

In [ ]:
import torch

audio_tensor = torch.load(audio_path)
audio_tensor, audio_tensor.size()

(tensor([[ 0.1969, -0.0129, -0.0882,  ...,  0.1390,  0.2094, -0.1379],
         [ 0.0950,  0.0111, -0.0779,  ...,  0.0576,  0.1827, -0.2337],
         [ 0.0894,  0.0031, -0.0777,  ...,  0.0161,  0.2006, -0.1645],
         ...,
         [-0.1300, -0.0172,  0.1780,  ...,  0.1942,  0.2564, -0.2170],
         [-0.0504, -0.0843,  0.2044,  ...,  0.3398,  0.2339, -0.1538],
         [ 0.0152, -0.0241,  0.1738,  ...,  0.4174,  0.0609, -0.0242]]),
 torch.Size([20679, 768]))

In [ ]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("temp.wav", temperature=0)
print(result["text"])

 Generally my greatest strength as an employee is my empathy. I'm very good at communicating with others because I'm very good at understanding where they're coming from. Even when they're upset, I can usually deal with people without getting to upset myself. And on the other side of that, my greatest weakness as an employee is that I'm kind of shy. I'm good at talking to people in a little community and to people that I don't really like to be very social like with my coworkers. But I'm good at dealing with customers.


In [1]:
from transformers import Wav2Vec2ForXVector, Wav2Vec2Processor

model_id = "facebook/wav2vec2-base"
model = Wav2Vec2ForXVector.from_pretrained(model_id, use_safetensors=True).to(
    "cpu"  # args.audio_model_device
)
processor = Wav2Vec2Processor.from_pretrained(model_id)
model.eval()

/home/solan/repos/airi-research/.venv/lib/python3.12/site-packages/transformers/configuration_utils.py:312: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of Wav2Vec2ForXVector were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'feature_extractor.bias', 'feature_extractor.weight', 'objective.weight', 'projector.bias', 'projector.weight', 'tdnn.0.kernel.bias', 'tdnn.0.kernel.weight', 'tdnn.1.kernel.bias', 'tdnn.1.kernel.weight', 'tdnn.2.kernel.bias', 'tdnn.2.kernel.weight', 'tdnn.3.kernel.bias', 'tdnn.3.kernel.weight', 'tdnn.4.kernel.bias', 'tdnn.4.kernel.weight']
You should probably TRAIN this model on a down-stream task to be able to

Wav2Vec2ForXVector(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (enco

In [2]:
import torchaudio

waveform, sr = torchaudio.load("temp.wav")
# waveform = waveform.to("cuda")
print(waveform.size())

torch.Size([2, 2367288])


In [23]:
signal = processor(waveform, sampling_rate=16000, return_tensors="pt")
signal.input_values.size()

torch.Size([1, 2, 2367288])

In [3]:
import sys

sys.getsizeof(waveform)

80

In [9]:
import torch

with torch.no_grad():
    outputs = model(waveform)

In [12]:
outputs.embeddings.size()

torch.Size([2, 512])

In [15]:
outputs.logits.size()

torch.Size([2, 512])

In [ ]:
# transformers.logging.set_verbosity_error()

# model_id = "facebook/wav2vec2-base"
# model = Wav2Vec2ForCTC.from_pretrained(
#     model_id, use_safetensors=True
# ).to(args.audio_model_device)
# processor = Wav2Vec2Processor.from_pretrained(model_id)
# model.eval()


# val_file_paths = sorted(VAL_DIR_PATH.glob("*.mp4"))

# emoti_eff: EmotiEffLibRecognizerBase = EmotiEffLibRecognizer(
#     model_name="enet_b0_8_best_vgaf",
#     device=args.video_model_device,
#     engine="torch",
# )


# features: list[np.ndarray] = []
# batch_size = 32
# for i in range(0, len(cropped_frames), batch_size):
#     batch = cropped_frames[i : i + batch_size]
#     batch_features = emoti_eff.extract_features(batch)
#     features.extend(batch_features)

# # boost performance of conversions list[np.ndarray] -> np.ndarray
# features = np.array(features)
# video_features: Float[torch.Tensor, "frames features"] = (  # noqa: F722
#     torch.from_numpy(features)
# )
# torch.save(
#     obj=video_features,
#     f=preprocessed_dir_path / "video" / f"{sample_path.stem}.pt",
# )